In [1]:
import torch, pickle, argparse, os, warnings, copy, time, mlflow
import numpy as np, pytorch_lightning as pl, matplotlib.pyplot as plt, eagerpy as ep
from models import ConvNet
from data_loader import load_test_data
from foolbox import PyTorchModel
from foolbox.attacks import LinfProjectedGradientDescentAttack
from foolbox.attacks.base import Repeated
from tqdm.notebook import tqdm
from attack_helper import run_batched_attack_cpu, batched_accuracy
from mlflow.tracking.artifact_utils import get_artifact_uri

In [2]:
def as_list(s):
    return list(map(int, s[1:-1].split(',')))

def get_hparam(df, run_id, param):
    return df[df['run_id']==run_id]['params.'+param].values[0]

In [3]:
tracking_uri = 'sqlite:///mlruns/database.db'
mlflow.set_tracking_uri(tracking_uri)
df=mlflow.search_runs(experiment_names=['model_training'])
print(df.keys())

Index(['run_id', 'experiment_id', 'status', 'artifact_uri', 'start_time',
       'end_time', 'metrics.test_loss', 'metrics.train_loss',
       'metrics.val_loss', 'metrics.epoch', 'metrics.train_acc',
       'metrics.loss', 'metrics.val_acc', 'metrics.test_acc',
       'params.es_stopped_epoch', 'params.batch_norm', 'params.channels',
       'params.train_batch_size', 'params.activation_fn', 'params.train_rot',
       'params.bandlimit', 'params.flat', 'params.es_mode',
       'params.es_min_delta', 'params.train_samples', 'params.es_monitor',
       'params.padded_img_size', 'params.nodes', 'params.name',
       'params.total_params', 'params.num_workers', 'params.weight_decay',
       'params.test_rot', 'params.kernel_max_beta', 'params.es_patience',
       'params.test_batch_size', 'params.lr', 'params.trainable_params',
       'params.max_epochs', 'params.strides', 'params.kernels',
       'tags.mlflow.user', 'tags.mlflow.runName', 'tags.model'],
      dtype='object')


In [4]:
df[['run_id', 'params.total_params']].head()

,run_id,params.total_params
0,b78b4173276841d48d777e57e51650f3,113943
1,ee3fdeb373ad4873a08ba4a0d48b2d4a,113943
2,fcbc6994b7764a02ae57170754d9eec8,113943
3,52f771e936494cad8a39fc8292758cf9,113943
4,7b18c11dda3943c9a385ee96645339f9,113943


In [5]:
df[df['params.total_params'] == str(113943)][['tags.mlflow.runName', 'metrics.test_acc', 'params.train_samples']]

,tags.mlflow.runName,metrics.test_acc,params.train_samples
104,1668152805,0.9920,50000
105,1668139362,0.9923,50000
106,1668127973,0.9915,50000
107,1668120367,0.9902,40000
108,1668110081,0.9911,40000
109,1668101504,0.9902,40000
110,1668094166,0.9888,30000
111,1668085318,0.9904,30000
112,1668077822,0.9895,30000
113,1668070267,0.9873,20000


In [10]:
df[df['tags.mlflow.runName']==str(1695204583)]['tags.model'].values[0]

'CConvNet'

In [4]:
eval(df[df['tags.mlflow.runName']==str(1695238653)]['params.test_rot'].values[0])

False

In [7]:
eval(df[df['tags.mlflow.runName']==str(1680863211)]['params.flat'].values[0])

True

In [11]:
print(df[df['tags.mlflow.runName']==str(1680863211)]['params.flat'].values[0])

True


In [16]:
eval(df[df['tags.mlflow.runName']==str(1680863211)]['params.padded_img_size'].values[0])

[60, 60]

In [4]:
df[df['tags.mlflow.runName']==str(1695204583)]['metrics.test_acc'].values[0]

0.920199990272522

In [5]:
df[df['tags.mlflow.runName']==str(1695238653)]['metrics.test_acc'].values[0]

0.9197999835014343

In [12]:
df[df['tags.mlflow.runName']==str(1686613889)]['metrics.test_acc'].values[0]

0.9933000206947327

In [6]:
run_id=df[df['tags.mlflow.runName']==str(1668139362)]['run_id'].values[0]

In [7]:
artifact_path = get_artifact_uri(run_id=run_id, tracking_uri=tracking_uri)

In [8]:
artifact_path

'./mlruns/2/d66eec77ff0044938ec96d1653a34355/artifacts'

In [ ]:
dirs=os.listdir(artifact_path)

In [ ]:
for s in dirs:
    if s.find('.ckpt') >= 0:
        checkpoint = s
        break

In [ ]:
checkpoint_path = os.path.join(artifact_path, checkpoint)

In [ ]:
best_model = torch.load(checkpoint_path)
hparams = argparse.Namespace(**best_model['hyper_parameters'])
model = ConvNet(hparams, None, None).eval()
model.load_state_dict(best_model['state_dict'])